# Estimating Density from Raw Trap Data

I'm taking in the trap counts per day across the trap grid as a 4 channel 16x16 image. 

In [ ]:
import mxnet as mx
from mxnet import nd, gluon, autograd
from mxboard import SummaryWriter
import torch

In [ ]:
# Hyper-parameters
batch_size = 100000
epochs = 1000
learning_rate = 0.01
trainperc = 0.8

context = mx.gpu()

In [ ]:
# Load data
simimages = torch.load("data/simimages.pt")
simimages = nd.array(simimages)

simdensities = torch.load("data/simdensities.pt")
simdensities = nd.array(simdensities)

numsamples = simdensities.shape[0]
num_train_samples = round(trainperc*numsamples)
num_test_samples = numsamples - round(trainperc*numsamples)

numbatches = num_train_samples / batch_size
numbatches

In [ ]:
train_data = gluon.data.DataLoader(gluon.data.ArrayDataset(simimages[:num_train_samples,:,:,:], 
                                                           simdensities[:num_train_samples]), 
                                   batch_size=batch_size, shuffle=True)
test_data = gluon.data.DataLoader(gluon.data.ArrayDataset(simimages[num_train_samples:,:,:,:], 
                                                          simdensities[num_train_samples:]), 
                                  batch_size=batch_size)

In [ ]:
class Reshape2D1D(gluon.Block):
    def __init__(self, **kwargs):
        super(Reshape2D1D, self).__init__(**kwargs)
        
    def forward(self, x):
        return x.reshape((x.shape[0],x.shape[1],256))

class Reshape1D2D(gluon.Block):
    def __init__(self, **kwargs):
        super(Reshape1D2D, self).__init__(**kwargs)
        
    def forward(self, x):
        return x.reshape((x.shape[0],x.shape[1],8,8))

In [ ]:
# Test layers (they reverse each other :) one example here)
# net = Reshape2D1D()
# net(x).shape

# oneway = gluon.nn.Sequential()
# oneway.add(Reshape2D1D())
# oneway.add(Reshape1D2D())

# (oneway(x) == x).min()

In [ ]:
# Define model
# net = gluon.nn.Sequential()

# with net.name_scope():
#     net.add(gluon.nn.Conv2D(channels=5, kernel_size=3, activation='relu'))
#     net.add(gluon.nn.Conv2D(channels=5, kernel_size=3, activation='relu'))
#     net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
#     net.add(gluon.nn.Conv2D(channels=5, kernel_size=3, activation='relu'))
#     net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
#     # The Flatten layer collapses all axis, except the first one, into one axis.
#     net.add(gluon.nn.Flatten())
#     net.add(gluon.nn.Dense(8, activation='relu'))
#     net.add(gluon.nn.Dense(8, activation='relu'))
#     net.add(gluon.nn.Dense(1))

# net

net = gluon.nn.Sequential()

with net.name_scope():
    net.add(Reshape2D1D())
    net.add(gluon.nn.Conv1D(channels=8, kernel_size=1, activation='relu'))
    net.add(gluon.nn.Conv1D(channels=8, kernel_size=1, activation='relu'))
    net.add(gluon.nn.MaxPool1D(pool_size=2))
    
    net.add(gluon.nn.Conv1D(channels=8, kernel_size=1, activation='relu'))
    net.add(gluon.nn.Conv1D(channels=8, kernel_size=1, activation='relu'))
    net.add(gluon.nn.MaxPool1D(pool_size=2))
    
    net.add(Reshape1D2D())
    net.add(gluon.nn.Conv2D(channels=4, kernel_size=3, activation='relu'))
    net.add(gluon.nn.Conv2D(channels=4, kernel_size=3, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    
    # The Flatten layer collapses all axis, except the first one, into one axis.
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(8, activation='relu'))
    net.add(gluon.nn.Dense(8, activation='relu'))
    net.add(gluon.nn.Dense(8, activation='relu'))
    net.add(gluon.nn.Dense(1))

net

In [ ]:
# Parameter initialization
net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=context)

In [ ]:
net.summary(nd.random.uniform(shape=(batch_size, 4, 16, 16), ctx=context))

In [ ]:
# Define optimizer and metrics
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': learning_rate})

metric_MSE = mx.metric.MSE() # train metric
loss_L2 = gluon.loss.L2Loss() # L2 loss

# define loop to test the model
def test(ctx):
    metric = mx.metric.MSE()
    for data, label in test_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        metric.update([label], [output])

    return metric.get()

In [ ]:
# define a summary writer that logs data and flushes to the file every 5 seconds
sw = SummaryWriter(logdir='./logs', flush_secs=5)

In [ ]:
# Train the model
global_step = 0
for e in range(epochs):
#     e += 1000
    # reset data iterator and metric at begining of epoch.
    metric_MSE.reset()
    
    # Loop through the training data
    for i, (data, label) in enumerate(train_data):
        # Copy data to context (ctx) if necessary
        data = data.as_in_context(context)
        label = label.as_in_context(context)
        # Start recording computation graph with record() section.
        # Recorded graphs can then be differentiated with backward.
        with autograd.record():
            output = net(data)
            L = loss_L2(output, label)
        
        sw.add_scalar(tag='train_loss', value=L.mean().asscalar(), global_step=global_step)
        global_step += 1
        L.backward()
        
        # take a gradient step with batch_size equal to data.shape[0]
        trainer.step(data.shape[0])
        
        # update metric at last.
        metric_MSE.update([label], [output])
        
    # logging training accuracy
    name, train_acc = metric_MSE.get()
    sw.add_scalar(tag='accuracy_curves', value=('train_acc', train_acc), global_step=e)
    
    # logging testing accuracy
    name, test_acc = test(context)
    sw.add_scalar(tag='accuracy_curves', value=('valid_acc', test_acc), global_step=e)
    
#     print("epoch:", e, "TrainMSE:", train_acc, "TestMSE:", test_acc)

In [ ]:
sw.export_scalars('scalar_dict.json')
sw.close()